In [105]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [106]:
import fooof
print(fooof.__version__)

1.0.1-dev


In [113]:
from pathlib import Path, PurePath
from glob import glob
import scipy.io  as sio
import numpy as np
import re
import xarray as xr
import pandas as pd

from lambic.spectrum import psd_fooof, fooof2pandas

pandas_list = []
folder ='../data/lemon_rsEEG_dataset'
files_EC = [Path(p) for p in glob(f"{folder}/*/*EC.mat")]

files_short = files_EC

data_arrays_128freq = []
data_arrays_52freq = []
sub_128freq = []
sub_52freq= []

for path_mat in files_short:
    sub = re.findall("sub-[0-9]{6}", str(path_mat))
    mat_content = sio.loadmat(path_mat)
    roi_names=  [l.flatten()[0] for l in mat_content["RowNames"].flatten()] #flatten
    freqs = np.ndarray.squeeze(mat_content["Freqs"])
    tf = mat_content["TF"]
    da = xr.DataArray(tf, dims=['roi_names', 'sub','freqs',], coords={'roi_names': roi_names, 'sub': sub,  'freqs': freqs})
    if freqs.size >  51:
        data_arrays_128freq.append(da)
        sub_128freq.append(sub)
    else:
        data_arrays_52freq.append(da)
        sub_52freq.append(sub)

psd_concat_freq128 = xr.concat(data_arrays_128freq , dim='sub', coords='all', join='override')
psd_concat_freq128.to_netcdf("data/freq128.nc")
display(f"psd_concat_freq128 is {len(sub_128freq)} elements")


psd_concat_freq52 = xr.concat(data_arrays_52freq  , dim='sub', coords='all', join='override')
psd_concat_freq52.to_netcdf("data/freq52.nc")
display(f"psd_concat_freq52 is {len(sub_52freq)} elements")




'psd_concat_freq128 is 199 elements'

'psd_concat_freq52 is 2 elements'